In [ ]:
ЗАДАЧИ

criterion = torch.nn.CrossEntropyLoss(ignore_index=pad_index)
0) Сделать бпе токенизацию
1) сделать один класс seq2seq
2) правильно рассчитывать лосс, не учитывая паддинг
правильно считать лосс, добавив токен bos (begin of sentence) в ту последовательность, 
которую подаешь в декодер и токен eos в таргет последовательность
3) впилить early stopping  и добавить больше эпох
и early stopping нужно впиливать аккуратно: сохранять лучшую модель, но и отдельно на каждой эпохе

что-то типа:
best_model_state_dict.pth и last_model_state_dict.pth

также хорошо бы сохранять еще state_dict оптимизаторов, так как в них хранятся статистики, на основе которых делаются 
все эти хитрые смещения типа моментума. сохранять стейт дикт оптимизатора нужно, если хочешь потом дообучать модель

это делается тоже просто
torch.save(seq2seq_optimizer.state_dict(), 'last_optimizer_state_dict.pth')
...

4) добавить фичи как в статье (позиционное кодирование, кодирование частей речи, кодирование слов, отвечающих на потенциальный вопрос)

In [1]:
!pip3 install torch
!pip3 install tqdm
!pip3 install nltk
!pip3 install pandas
!pip3 install numpy
!pip3 install seaborn
!apt-get install wget

     |████████████████████████████████| 734.6MB 68kB/s s eta 0:00:01     |███████████████████████▏        | 531.5MB 63.7MB/s eta 0:00:04     |██████████████████████████████  | 689.8MB 62.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 791kB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 1.1MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1450714 sha256=d442d3def43b67d4b1a64b9e01fe9ae1ad49827f908c4e41ca1ad147ddda82d5
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
     |████████████████████████████████| 10.4MB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 512kB 51.3MB/s eta 0:00:01
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  wget
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 316 kB of archives.
After

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from tqdm.auto import tqdm

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm.auto import tqdm

import pandas as pd

import numpy as np

from torch.utils.data import Dataset, DataLoader

In [2]:
import math
import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from matplotlib import pyplot as plt

In [3]:
#ИЗМЕНИТЬ ПУТЬ В ЗАВИСИМОСТИ ОТ ТОГО ГДЕ ЗАПУСКАЕМ КОД
try:
    df = pd.read_csv("ctx_quest.csv")
except:
    df = pd.read_csv("/Users/lilyakhoang/input/question_generation/ctx_quest.csv")

df.head()

,Unnamed: 0,context,question
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?
1,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...
2,2,"Managed by her father, Mathew Knowles, the gro...",When did Beyonce leave Destiny's Child and bec...
3,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?
4,4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86821 entries, 0 to 86820
Data columns (total 3 columns):
Unnamed: 0    86821 non-null int64
context       86610 non-null object
question      86821 non-null object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [5]:
df_nonan = df.dropna()
df_nonan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86610 entries, 0 to 86820
Data columns (total 3 columns):
Unnamed: 0    86610 non-null int64
context       86610 non-null object
question      86610 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ MB


In [6]:
# для "обучения" bpe модели нам нужно сохранить данные для обучения в отдельный файл
# где будут построчно храниться тексты
err = 0
with open('/Users/lilyakhoang/input/for_bpe_ctx_quest.txt', 'w') as f:
    for que in df_nonan.context:
        try:
            f.write(que + '\n')
        except:
            err += 1
    for que in df_nonan.question:
        try:
            f.write(que + '\n')
        except:
            err += 1
print(err)

0


In [7]:
import youtokentome as yttm

In [8]:
# обучаем
vocab_size = 16000
model_path = 'bpe.model'

yttm.BPE.train(data='/Users/lilyakhoang/input/for_bpe_ctx_quest.txt', vocab_size=vocab_size, model=model_path)

In [9]:
tokenizer = yttm.BPE(model=model_path)

In [10]:
tokenizer.encode(list(df_nonan.context[:2]), bos=True, eos=True, output_type=yttm.OutputType.SUBWORD)

[['<BOS>',
  '▁Beyoncé',
  '▁G',
  'is',
  'elle',
  '▁Know',
  'les',
  '-C',
  'arter',
  '▁(/',
  'bi',
  'ː',
  'ˈ',
  'j',
  'ɒ',
  'n',
  'se',
  'ɪ',
  '/',
  '▁be',
  'e',
  '-',
  'Y',
  'O',
  'N',
  '-s',
  'ay',
  ')',
  '▁(b',
  'orn',
  '▁September',
  '▁4,',
  '▁198',
  '1)',
  '▁is',
  '▁an',
  '▁American',
  '▁sing',
  'er,',
  '▁song',
  'writ',
  'er,',
  '▁record',
  '▁producer',
  '▁and',
  '▁act',
  'ress.',
  '▁B',
  'orn',
  '▁and',
  '▁raised',
  '▁in',
  '▁Hou',
  'ston,',
  '▁Texas,',
  '▁she',
  '▁performed',
  '▁in',
  '▁various',
  '▁singing',
  '▁and',
  '▁d',
  'ancing',
  '▁competitions',
  '▁as',
  '▁a',
  '▁child,',
  '▁and',
  '▁rose',
  '▁to',
  '▁fame',
  '▁in',
  '▁the',
  '▁late',
  '▁1990s',
  '▁as',
  '▁lead',
  '▁singer',
  '▁of',
  '▁R&B',
  '▁girl',
  '-g',
  'roup',
  "▁Destiny's",
  '▁Child',
  '.',
  '▁Man',
  'aged',
  '▁by',
  '▁her',
  '▁father,',
  '▁Mat',
  'hew',
  '▁Know',
  'les,',
  '▁the',
  '▁group',
  '▁became',
  '▁one',
  '▁

In [11]:
# давайте токенизируем наш датасет
# токенизирую батчами, потому что так быстрее
# также в начало добавляем токен bos (begin of sentence)

tokenized_ctx = []
tokenized_quest = []
batch_size = 256

for i_batch in tqdm(range(math.ceil(len(df_nonan.context) / batch_size))):
    
    tokenized_ctx.extend(tokenizer.encode(list(df_nonan.context[i_batch*batch_size:(i_batch+1)*batch_size]), bos=True))

for i_batch in tqdm(range(math.ceil(len(df_nonan.question) / batch_size))):

    tokenized_quest.extend(tokenizer.encode(list(df_nonan.question[i_batch*batch_size:(i_batch+1)*batch_size]), bos = True, eos=True))



100%|██████████| 339/339 [00:01<00:00, 194.20it/s]


In [28]:
tokenized_quest[1]

[2, 1411, 2461, 1503, 7635, 9910, 1305, 1795, 2402, 1361, 5194, 1761, 36, 3]

In [29]:
tokenizer.vocab()[:15]

['<PAD>',
 '<UNK>',
 '<BOS>',
 '<EOS>',
 '▁',
 'e',
 't',
 'a',
 'i',
 'n',
 'o',
 'r',
 's',
 'h',
 'l']

In [12]:
class WordData(torch.utils.data.Dataset):
    def __init__(self, context_list, questions_list, context_len, questions_len, pad_index, eos_index):
        self.context_list = context_list
        self.questions_list = questions_list
        
        self.context_len = context_len
        self.questions_len = questions_len
        
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.context_list)
    
    def __getitem__(self, index):
        
        context = self.context_list[index][:self.context_len]
        pads_ctx = [self.pad_index] * (self.context_len - len(context))
#         print(len(pads_ctx))
        context = torch.tensor(context + pads_ctx).long()
        
        question = self.questions_list[index][:self.questions_len]
        pads_quest = [self.pad_index] * (self.questions_len - len(question))
        question = torch.tensor(question + pads_quest).long()
        
        return context, question

In [15]:
tokenizer.vocab()[pad_index]

'<PAD>'

In [14]:
batch_size = 64

context_len = 80
quest_len = 20

pad_index = 0
eos_index = 3

In [16]:
validation_start_index = int(len(tokenized_ctx) * 0.05)

In [17]:
train_dataset = WordData(context_list=tokenized_ctx[:-validation_start_index], questions_list = tokenized_quest[:-validation_start_index],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

validation_dataset = WordData(context_list=tokenized_ctx[-validation_start_index:],questions_list = tokenized_quest[-validation_start_index:],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

len(train_dataset), len(validation_dataset)

(82280, 4330)

In [18]:
train_dataset.__getitem__(0)

(tensor([    2,  3297,  1394,  1304, 13539, 12243,  1838,  4502,  4113,  8059,
         13895,   100,    90,    65,   197,     9,  1444,   112,    76,  1348,
             5,    42,    73,    62,    51,  2250,  1398,    47,  4690,  2117,
          2811,  6982,  2156,  8762,  1352,  1306,  1959,  2339,  2087,  3046,
          7582,  2087,  2083,  7536,  1312,  1875, 11109,  1365,  2117,  1312,
          5267,  1305,  4274, 13461, 15846,  2402,  4829,  1305,  2734, 10861,
          1312,  1317,  9041, 12046,  1363,  1285, 12745,  1312,  7309,  1319,
         13738,  1305,  1287,  2535,  7915,  1363,  2018,  8292,  1300,  9914]),
 tensor([   2, 1800, 1503, 7635, 2294, 4596, 2454,   36,    3,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]))

In [19]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=2)

In [20]:
for x, y in train_loader:
    print(x[0],y[0])
    break

tensor([    2,  3297,  1394,  1304, 13539, 12243,  1838,  4502,  4113,  8059,
        13895,   100,    90,    65,   197,     9,  1444,   112,    76,  1348,
            5,    42,    73,    62,    51,  2250,  1398,    47,  4690,  2117,
         2811,  6982,  2156,  8762,  1352,  1306,  1959,  2339,  2087,  3046,
         7582,  2087,  2083,  7536,  1312,  1875, 11109,  1365,  2117,  1312,
         5267,  1305,  4274, 13461, 15846,  2402,  4829,  1305,  2734, 10861,
         1312,  1317,  9041, 12046,  1363,  1285, 12745,  1312,  7309,  1319,
        13738,  1305,  1287,  2535,  7915,  1363,  2018,  8292,  1300,  9914]) tensor([   2, 1800, 1503, 7635, 2294, 4596, 2454,   36,    3,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])


In [21]:
# пробежимся по итератору, чтобы убедиться что ничего не падает и он работает достаточно быстро

progress_bar = tqdm(total=len(train_loader.dataset), desc='Testing')

for x, y in train_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing: 100%|██████████| 82280/82280 [00:02<00:00, 28053.00it/s]


In [22]:
# пробежимся по итератору, чтобы убедиться что ничего не падает и он работает достаточно быстро

progress_bar = tqdm(total=len(validation_loader.dataset), desc='Testing')

for x, y in validation_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing: 100%|██████████| 4330/4330 [00:00<00:00, 26364.34it/s]


In [ ]:
# https://medium.com/@adam.wearne/seq2seq-with-pytorch-46dc00ff5164

In [ ]:
# class Seq2Seq(nn.Module):
#     def __init__(self, vocab_len, hidden_size):
#         super(Seq2Seq, self).__init__()
#         self.hidden_size = hidden_size
#         self.vocab_len = vocab_len
        
#         #encoder
#         self.encoder_embedding = nn.Embedding(self.vocab_len, hidden_size)
#         self.encoder_gru = nn.GRU(hidden_size, hidden_size)
        
#         #decoder 
#         self.decoder_embedding = nn.Embedding(self.hidden_size, vocab_len)
#         self.decoder_gru = nn.GRU(hidden_size, hidden_size)
#         self.decoder_out = nn.Linear(hidden_size, output_size)
#         self.decoder_softmax = nn.LogSoftmax(dim=1)
    
#     def forward(input_sequence, output_sequence):
#         output = embedded
        
#         decoder_input = torch.tensor([[SOS_token]], device=device)
        
#         encoder_outputs, hidden = self.encoder(input_tokens, input_lengths)
        


In [ ]:
pad_idx = TEXT.vocab.stoi['<pad>']
eos_idx = TEXT.vocab.stoi['<eos>']
sos_idx = TEXT.vocab.stoi['<sos>']

In [77]:
len(tokenizer.vocab())

16000

In [75]:
tokenizer.vocab()[:10]

['<PAD>', '<UNK>', '<BOS>', '<EOS>', '▁', 'e', 't', 'a', 'i', 'n']

In [76]:
tokenizer.vocab().index("<PAD>")

0

In [92]:

# pretrained_embeddings = TEXT.vocab.vectors
# model.embedding.weight.data.copy_(pretrained_embeddings)
# UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
# model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
# model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)
# model.embedding.weight.requires_grad = False
# optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True]
#                        , lr=1.0e-3)
# criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

In [96]:
# пробежимся по итератору, чтобы убедиться что ничего не падает и он работает достаточно быстро

progress_bar = tqdm(total=len(train_loader.dataset), desc='Testing')

for x, y in train_loader:
    print(x[0],y[0])
    progress_bar.update(x.size(0))
    break
    
progress_bar.close()

Testing:   0%|          | 64/82280 [00:00<00:11, 7274.48it/s]

tensor([    2,  3297,  1394,  1304, 13539, 12243,  1838,  4502,  4113,  8059,
        13895,   100,    90,    65,   197,     9,  1444,   112,    76,  1348,
            5,    42,    73,    62,    51,  2250,  1398,    47,  4690,  2117,
         2811,  6982,  2156,  8762,  1352,  1306,  1959,  2339,  2087,  3046,
         7582,  2087,  2083,  7536,  1312,  1875, 11109,  1365,  2117,  1312,
         5267,  1305,  4274, 13461, 15846,  2402,  4829,  1305,  2734, 10861,
         1312,  1317,  9041, 12046,  1363,  1285, 12745,  1312,  7309,  1319,
        13738,  1305,  1287,  2535,  7915,  1363,  2018,  8292,  1300,  9914]) tensor([   2, 1800, 1503, 7635, 2294, 4596, 2454,   36,    3,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])


In [23]:
def evaluate (model, iterator):
    epoch_loss = 0
    for idx, batch in tqdm(enumerate(iterator), total=len(iterator)):
        with torch.no_grad():
            input_seq = batch[0]
            target_tokens = batch[1]
            output = model(input_seq, target_tokens)
            output = output.view(-1, output.size(-1))
            target_tokens = target_tokens.view(-1)
            print("AFTER RESHAPE PREDICTED_OUTPUT {}, REAL_OUTPUT {}".format(output.shape, target_tokens.shape))
            loss = criterion(output, target_tokens)
            epoch_loss += loss.item()
        break
    return epoch_loss / len(iterator)

In [29]:
def train(model, iterator, criterion, optimizer):
   # Put the model in training mode!
    model.train()
    
    epoch_loss = 0
    losses_list = []
    
    for idx, batch in tqdm(enumerate(iterator), total=len(iterator)):
        optimizer.zero_grad()
        input_seq = batch[0]
        target_tokens = batch[1]
#         print(len(batch), batch[0][0],batch[1][0])
        output = model(input_seq, target_tokens)
        
        print("PREDICTED_OUTPUT {}, REAL_OUTPUT {}".format(output.shape, target_tokens.shape))
#         output = output.view(-1)
        output = output.view(-1, output.size(-1))
        target_tokens = target_tokens.view(-1)
        print("AFTER RESHAPE PREDICTED_OUTPUT {}, REAL_OUTPUT {}".format(output.shape, target_tokens.shape))
        loss = criterion(output, target_tokens)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        losses_list.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses_list[-100:]))
        break
    return epoch_loss / len(iterator)

In [28]:
# MY VERSIOB
pad_idx = tokenizer.vocab().index("<PAD>")
eos_idx = tokenizer.vocab().index("<EOS>")
sos_idx = tokenizer.vocab().index("<BOS>")
# Size of embedding_dim should match the dim of pre-trained word embeddings!
embedding_dim = 300
hidden_dim = 512
vocab_size = len(tokenizer.vocab())
model = My_Seq2Seq(embedding_dim,
                 hidden_dim, 
                 vocab_size, 
                 device, pad_idx, eos_idx, sos_idx).to(device)
optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=1.0e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

validation_losses =[]
train_losses = []

N_EPOCHS = 20
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, criterion, optimizer)
    train_losses.append(train_loss)
    
    if min(train_losses) == train_loss and len(train_losses) > 1:
        torch.save(model.state_dict, "best_seq2seq")
        torch.save(optimizer.state_dict, "best_Adam_state_dict")
#     print("train_loss",train_loss)
    
    torch.save(model.state_dict, "last_seq2seq")
    torch.save(optimizer.state_dict, "Adam_state_dict")
    #early stopping
    test_loss = evaluate(model, validation_loader)
    validation_losses.append(test_loss)
    if len(validation_losses) > 1 and validation_losses[epoch] > validation_losses[epoch-1]:
        print("stop")
        break
#     print("test_loss",test_loss)

#     break

  0%|          | 0/41140 [00:00<?, ?it/s]

torch.Size([2, 20, 16000])

 Start handling new element
target_tensor.shape torch.Size([20]), target_tensor tensor([   2, 1800, 1503, 7635, 2294, 4596, 2454,   36,    3,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
go_to_decoder_with decoder_input tensor([[2]]), decoder_hidden.shape torch.Size([1, 1, 512])
iterate_over_target 0th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[2]])
iterate_over_target 1th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[1800]])
iterate_over_target 2th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[1503]])
iterate_over_target 3th time, decoder_input.shape tor


/Users/lilyakhoang/anaconda3/lib/python3.7/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type My_Seq2Seq. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/lilyakhoang/anaconda3/lib/python3.7/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type EncoderRNN_inside_class. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/lilyakhoang/anaconda3/lib/python3.7/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type DecoderRNN_inside_class. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  0%|          | 0/41140 [00:00<?, ?it/s]

torch.Size([2, 20, 16000])

 Start handling new element
target_tensor.shape torch.Size([20]), target_tensor tensor([    2,  1800,  1503,  9346,  2453,  1287,  5133,  4208,  6387,  1319,
        11858,    36,     3,     0,     0,     0,     0,     0,     0,     0])
go_to_decoder_with decoder_input tensor([[2]]), decoder_hidden.shape torch.Size([1, 1, 512])
iterate_over_target 0th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[2]])
iterate_over_target 1th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[1800]])
iterate_over_target 2th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[1503]])
iterate_over_target 3th time, dec


  0%|          | 0/41140 [00:00<?, ?it/s]

...


  0%|          | 0/41140 [00:00<?, ?it/s]

torch.Size([2, 20, 16000])

 Start handling new element
target_tensor.shape torch.Size([20]), target_tensor tensor([    2,  1800,  1503,  9346,  2453,  1287,  5133,  4208,  6387,  1319,
        11858,    36,     3,     0,     0,     0,     0,     0,     0,     0])
go_to_decoder_with decoder_input tensor([[2]]), decoder_hidden.shape torch.Size([1, 1, 512])
iterate_over_target 0th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[2]])
iterate_over_target 1th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[1800]])
iterate_over_target 2th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[1503]])
iterate_over_target 3th time, dec


  0%|          | 0/41140 [00:00<?, ?it/s]

torch.Size([2, 20, 16000])

 Start handling new element
target_tensor.shape torch.Size([20]), target_tensor tensor([    2,  1800,  1503,  9346,  2453,  1287,  5133,  4208,  6387,  1319,
        11858,    36,     3,     0,     0,     0,     0,     0,     0,     0])
go_to_decoder_with decoder_input tensor([[2]]), decoder_hidden.shape torch.Size([1, 1, 512])
iterate_over_target 0th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[2]])
iterate_over_target 1th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[1800]])
iterate_over_target 2th time, decoder_input.shape torch.Size([1, 1])
AFTER_DECODER_STEP decoder_output.shape torch.Size([1, 16000]), decoder_input.shape torch.Size([1, 1]), decoder_input tensor([[1503]])
iterate_over_target 3th time, dec

KeyboardInterrupt: 

In [ ]:
def DEFAULT_train(model, iterator, criterion, optimizer, clip=1.0):
   # Put the model in training mode!
   model.train()
    
    epoch_loss = 0
    
    for idx, batch in tqdm(enumerate(iterator), total=len(iterator)):
        
        input_sequence = batch.input_sequence
        output_sequence = batch.output_sequence
        
        target_tokens = output_sequence[0]
        
        # zero out the gradient for the current batch
        optimizer.zero_grad()
        
        # Run the batch through our model
        output = model(input_sequence, output_sequence)
        
        # Throw it through our loss function
        output = output[1:].view(-1, output.shape[-1])
        target_tokens = target_tokens[1:].view(-1)
        
        loss = criterion(output, target_tokens)
        
        # Perform back-prop and calculate the gradient of our loss function
        loss.backward()
          
        # Clip the gradient if necessary.          
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # Update model parameters
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [25]:
class My_Seq2Seq(nn.Module):
    def __init__(self, embedding_size, hidden_size, vocab_size, 
                 device, pad_idx, eos_idx, sos_idx):
        super(My_Seq2Seq, self).__init__()
        
        self.vocab_size = vocab_size
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        # Encoder network
        self.encoder = EncoderRNN_inside_class(hidden_size, 
                               embedding_size, 
                               self.embedding)
        
        # Decoder network        
        self.decoder = DecoderRNN_inside_class(self.embedding,
                               embedding_size,
                              hidden_size,
                              vocab_size)
        
        self.pad_idx = pad_idx
        self.eos_idx = eos_idx
        self.sos_idx = sos_idx
        self.device = device
    def forward(self, input_sequence, output_sequence, debug = True):
        input_tokens = input_sequence[0]
        input_lengths = input_sequence[1]
        
        encoder_hidden = self.encoder.initHidden(len(input_sequence))
        encoder_output, encoder_hidden = self.encoder(input_sequence, encoder_hidden)
        batch_size = len(input_sequence)
        outputs = torch.zeros(batch_size, 20, self.vocab_size).to(self.device)
        print(outputs.shape)
        for batch_element_index in range(batch_size):
            target_tensor = output_sequence[batch_element_index, :]
            if debug == True: 
                print("\n Start handling new element")
                print("target_tensor.shape {}, target_tensor {}".format(target_tensor.shape, target_tensor))

            decoder_input = torch.tensor([[self.sos_idx]], device=device)
            decoder_hidden = encoder_hidden[:,batch_element_index,:].unsqueeze(1)

            if debug == True: 
                print("go_to_decoder_with decoder_input {}, decoder_hidden.shape {}".format(decoder_input, decoder_hidden.shape))
            
            # Teacher forcing: Feed the target as the next input
            for di in range(20):
                if debug == True: 
                    print("iterate_over_target {}th time, decoder_input.shape {}".format(di, decoder_input.shape))
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                outputs[batch_element_index, di, :] = decoder_output
                decoder_input = torch.tensor([target_tensor[di]], device=device).unsqueeze(1)
                if debug == True: 
                    print("AFTER_DECODER_STEP decoder_output.shape {}, decoder_input.shape {}, decoder_input {}".format(decoder_output.shape, decoder_input.shape, decoder_input))
                if decoder_input[0][0] == 0:
                    break
        return outputs

In [26]:
class EncoderRNN_inside_class(nn.Module):
    def __init__(self, hidden_size, embedding_size, embedding):#input_size
        super(EncoderRNN_inside_class, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(embedding_size, hidden_size, batch_first = True)

    def forward(self, input, hidden, debug = False):
        if debug == True: 
            print("====ENCODING_FORWARD====")
            print("input.shape", input.shape)
#         embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.embedding(input)
        output = embedded
        if debug == True: 
            print("embedded/output.shape",embedded.shape,"hidden.shape", hidden.shape  )
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [27]:
class DecoderRNN_inside_class(nn.Module):
    
    def __init__(self, embedding, embedding_size,
                 hidden_size, output_size):
        super(DecoderRNN_inside_class, self).__init__()
        self.hidden_size = hidden_size
        self.vectors = embedding
        self.embedding = embedding
        self.gru = nn.GRU(embedding_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, debug = False):
        if debug == True:
          print("===FORWARD_DECODER===")
          print("input.shape {}, hidden.shape {}".format(input.shape,hidden.shape ))
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
pad_idx = TEXT.vocab.stoi['<pad>']
eos_idx = TEXT.vocab.stoi['<eos>']
sos_idx = TEXT.vocab.stoi['<sos>']
# Size of embedding_dim should match the dim of pre-trained word embeddings!
embedding_dim = 300
hidden_dim = 512
vocab_size = len(TEXT.vocab)
model = seq2seq(embedding_dim,
                 hidden_dim, 
                 vocab_size, 
                 device, pad_idx, eos_idx, sos_idx).to(device)
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)
model.embedding.weight.requires_grad = False
optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True]
                       , lr=1.0e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

In [ ]:
class DEFAULT_seq2seq(nn.Module):
    def __init__(self, embedding_size, hidden_size, vocab_size, 
                 device, pad_idx, eos_idx, sos_idx, teacher_forcing_ratio=0.5):
        super(seq2seq, self).__init__()
        
        # Embedding layer shared by encoder and decoder
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        # Encoder network
        self.encoder = Encoder(hidden_size, 
                               embedding_size, 
                               self.embedding,
                              num_layers=2,
                              dropout=0.5)
        
        # Decoder network        
        self.decoder = Decoder(self.embedding,
                               embedding_size,
                              hidden_size,
                              vocab_size,
                              n_layers=2,
                              dropout=0.5)
        
        
        # Indices of special tokens and hardware device 
        self.pad_idx = pad_idx
        self.eos_idx = eos_idx
        self.sos_idx = sos_idx
        self.device = device
        
    def create_mask(self, input_sequence):
        return (input_sequence != self.pad_idx).permute(1, 0)
        
        
    def forward(self, input_sequence, output_sequence, teacher_forcing_ratio=0.5):
      
        # Unpack input_sequence tuple
        input_tokens = input_sequence[0]
        input_lengths = input_sequence[1]
      
        # Unpack output_tokens, or create an empty tensor for text generation
        if output_sequence is None:
            inference = True
            output_tokens = torch.zeros((100, input_tokens.shape[1])).long().fill_(self.sos_idx).to(self.device)
        else:
            inference = False
            output_tokens = output_sequence[0]
vocab_size = self.decoder.output_size
        
        batch_size = len(input_lengths)
        max_seq_len = len(output_tokens)
        
        
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_seq_len, batch_size, vocab_size).to(self.device)
        
        
        # Pass through the first half of the network
        encoder_outputs, hidden = self.encoder(input_tokens, input_lengths)
        
        # Ensure dim of hidden_state can be fed into Decoder
        hidden =  hidden[:self.decoder.n_layers]
        
        #first input to the decoder is the <sos> tokens
        output = output_tokens[0,:]
        
        # Create mask
        mask = self.create_mask(input_tokens)
        
        
        # Step through the length of the output sequence one token at a time
        # Teacher forcing is used to assist training
        for t in range(1, max_seq_len):
            output = output.unsqueeze(0)
            
            output, hidden = self.decoder(output, hidden, encoder_outputs, mask)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (output_tokens[t] if teacher_force else top1)
            
            # If we're in inference mode, keep generating until we produce an
            # <eos> token
            if inference and output.item() == self.eos_idx:
                return outputs[:t]
return outputs

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, vectors=vectors):#input_size
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.vectors = vectors
        self.vocab_size, self.embedding_dim = vectors.shape
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(self.vectors))
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first = True)

    def forward(self, input, hidden, debug = False):
        if debug == True: 
            print("====ENCODING_FORWARD====")
            print("input.shape", input.shape)
#         embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.embedding(input)
        output = embedded
        if debug == True: 
            print("embedded/output.shape",embedded.shape,"hidden.shape", hidden.shape  )
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 64, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, vectors):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.vectors = vectors
        self.vocab_size, self.embedding_dim = vectors.shape
        # self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(self.vectors))
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, self.vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, debug = False):
        if debug == True:
          print("===FORWARD_DECODER===")
          print("input.shape {}, hidden.shape {}".format(input.shape,hidden.shape ))
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class WordData(torch.utils.data.Dataset):
    
    def __init__(self, data, max_len, pad_index, eos_index):
        
        self.data = data
        
        self.max_len = max_len
        
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        sequence = self.data[index][:self.max_len]
        
        # исходная последовательность
        x = sequence[:]
        # нужно предсказать смещенную последовательность
        y = sequence[1:] + [self.eos_index]
        
        assert len(x) == len(y)
        
        pads = [self.pad_index] * (self.max_len - len(x))
        
        x = torch.tensor(x + pads).long()
        y = torch.tensor(y + pads).long()
        
        return x, y

In [31]:
class WordData(Dataset):
    
    def __init__(self, ctx_quest_pairs_list , word2index, context_sequence_length=80, 
                 question_sequence_length = 20, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = []
        
        self.word2index = word2index
        self.context_sequence_length = context_sequence_length
        self.question_sequence_length = question_sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(ctx_quest_pairs_list, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
    
        words = wordpunct_tokenize(text.lower())

        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for ctx_quest_pair in data_iterator:
            ctx = ctx_quest_pair[0]
            quest = ctx_quest_pair[1]

            ctx = self.process_text(ctx)
            indexed_ctx = self.indexing(ctx)
            self.x_data.append(indexed_ctx)

            quest = self.process_text(quest)
            indexed_quest = self.indexing(quest)
            self.y_data.append(indexed_quest)
    
    def indexing(self, tokenized_text):
        indexes = []
        for word in tokenized_text:
          if word in self.word2index:
            indexes.append(self.word2index[word])
        return indexes
    
    def padding(self, sequence, sequence_type):
        if sequence_type == 'context':
          sequence_length = self.context_sequence_length
        elif sequence_type == "question":
          sequence_length = self.question_sequence_length
        else:
          raise "unkown sequence type"
        count = 0 
        paded_seq = []
        for seq_el in sequence:
          paded_seq.append(seq_el)
          count += 1
          if count >= sequence_length: break
        if count < sequence_length:
          for ind in range (count, sequence_length):
            paded_seq.append(self.pad_index)

        return paded_seq
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x, "context")
        x = torch.Tensor(x).long()

        y = self.y_data[idx]
        y = self.padding(y, "question")
        y = torch.Tensor(y).long()
        
        return x, y

In [4]:
# потом можете добавить свою предобработку

def process_text(text):
    
    words = wordpunct_tokenize(text.lower())
    
    return words

In [5]:
all_text_data = []
all_text_data.extend(list(df['context']))
all_text_data.extend(list(df['question']))
all_text_data = list(set(all_text_data))
cleaned_text_data = [x for x in all_text_data if 'float' not in str(type(x))]
cleaned_text_data[:2]

['Who did Antipater declar as his successor?',
 'Which Tennessee Senator was the only Republican first-time Senator elected in 2006?']

In [6]:
word2freq = {}
lengths = []

for text in tqdm(cleaned_text_data):
  
    words = process_text(text)
    
    lengths.append(len(words))
    
    for word in words:
        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1

In [7]:
import seaborn as sns
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

In [ ]:

plt.figure(figsize=(16, 10))
plt.title('Распределение длин слов в текстах')
plt.xlabel('Длина предложения')
plt.ylabel('Доля')
sns.distplot(lengths)

In [7]:
upper_threshold = 80
lower_threshold = 10

correct_percent = len([sent_len for sent_len in lengths 
                       if sent_len <= upper_threshold and sent_len >= lower_threshold]) * 100 / len(lengths)

'{:.2f} % наших текстов входят в промежуток от {} до {} слов'.format(correct_percent, lower_threshold, upper_threshold)

'66.57 % наших текстов входят в промежуток от 10 до 80 слов'

In [8]:
SENTENCE_LENGTH = 80
QUESTION_LENGTH = 20

In [9]:
# #ЗАПУСТИТЬ ЕСЛИ ИЗ КОЛАБА
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

In [10]:
local_path = "/Users/lilyakhoang/input/glove.6B/glove.6B.50d.txt"
this_folder_path = "glove.6B.50d.txt"
with open(local_path, "r") as lines:
    for line in lines:
      print(line.split()[0], line.split()[0] in word2freq, len(line.split()[1:]))
      print(line.split()[1:])
      break


the True 50
['0.418', '0.24968', '-0.41242', '0.1217', '0.34527', '-0.044457', '-0.49688', '-0.17862', '-0.00066023', '-0.6566', '0.27843', '-0.14767', '-0.55677', '0.14658', '-0.0095095', '0.011658', '0.10204', '-0.12792', '-0.8443', '-0.12181', '-0.016801', '-0.33279', '-0.1552', '-0.23131', '-0.19181', '-1.8823', '-0.76746', '0.099051', '-0.42125', '-0.19526', '4.0071', '-0.18594', '-0.52287', '-0.31681', '0.00059213', '0.0074449', '0.17778', '-0.15897', '0.012041', '-0.054223', '-0.29871', '-0.15749', '-0.34758', '-0.045637', '-0.44251', '0.18785', '0.0027849', '-0.18411', '-0.11514', '-0.78581']


In [12]:
#ИЗМЕНИТЬ ПУТЬ В ЗАВИСИМОСТИ ОТ ТОГО ГДЕ ЗАПУСКАЕМ КОД
local_path = "/Users/lilyakhoang/input/glove.6B/glove.6B.50d.txt"
this_folder_path = "glove.6B.50d.txt"

word2index = {'PAD': 0}
vectors = []
   
try:
    lines = open(local_path)
except:
    lines = open(this_folder_path)
    
embedding_dim = 50
# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))
progress_bar = tqdm(desc='Read word2vec', total=400000)

for line in lines:
    current_word = line.split()[0]
    if current_word in word2freq:

        word2index[current_word] = len(word2index)

        # current_vectors = current_parts[-embedding_dim:]
        current_vectors = line.split()[1:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)

        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()

vectors = np.concatenate(vectors)

lines.close()

In [13]:
unk_words = [word for word in word2freq if word not in word2index]
unk_counts = [word2freq[word] for word in unk_words]
n_unk = sum(unk_counts) * 100 / sum(list(word2freq.values()))

sub_sample_unk_words = {word: word2freq[word] for word in unk_words}
sorted_unk_words = list(sorted(sub_sample_unk_words, key=lambda x: sub_sample_unk_words[x], reverse=True))

print('Мы не знаем {:.2f} % слов в датасете'.format(n_unk))
print('Количество неизвестных слов {} из {}, то есть {:.2f} % уникальных слов в словаре'.format(
    len(unk_words), len(word2freq), len(unk_words) * 100 / len(word2freq)))
print('В среднем каждое встречается {:.2f} раз'.format(np.mean(unk_counts)))
print()
print('Топ 5 невошедших слов:')

for i in range(5):
    print(sorted_unk_words[i], 'с количеством вхождениий -', word2freq[sorted_unk_words[i]])

Мы не знаем 1.31 % слов в датасете
Количество неизвестных слов 15247 из 80148, то есть 19.02 % уникальных слов в словаре
В среднем каждое встречается 4.49 раз

Топ 5 невошедших слов:
), с количеством вхождениий - 8360
). с количеством вхождениий - 6262
". с количеством вхождениий - 2903
", с количеством вхождениий - 2574
." с количеством вхождениий - 2351


In [14]:
class WordData(Dataset):
    
    def __init__(self, ctx_quest_pairs_list , word2index, context_sequence_length=80, 
                 question_sequence_length = 20, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = []
        
        self.word2index = word2index
        self.context_sequence_length = context_sequence_length
        self.question_sequence_length = question_sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(ctx_quest_pairs_list, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
    
        words = wordpunct_tokenize(text.lower())

        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for ctx_quest_pair in data_iterator:
            ctx = ctx_quest_pair[0]
            quest = ctx_quest_pair[1]

            ctx = self.process_text(ctx)
            indexed_ctx = self.indexing(ctx)
            self.x_data.append(indexed_ctx)

            quest = self.process_text(quest)
            indexed_quest = self.indexing(quest)
            self.y_data.append(indexed_quest)
    
    def indexing(self, tokenized_text):
        indexes = []
        for word in tokenized_text:
          if word in self.word2index:
            indexes.append(self.word2index[word])
        return indexes
    
    def padding(self, sequence, sequence_type):
        if sequence_type == 'context':
          sequence_length = self.context_sequence_length
        elif sequence_type == "question":
          sequence_length = self.question_sequence_length
        else:
          raise "unkown sequence type"
        count = 0 
        paded_seq = []
        for seq_el in sequence:
          paded_seq.append(seq_el)
          count += 1
          if count >= sequence_length: break
        if count < sequence_length:
          for ind in range (count, sequence_length):
            paded_seq.append(self.pad_index)

        return paded_seq
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x, "context")
        x = torch.Tensor(x).long()

        y = self.y_data[idx]
        y = self.padding(y, "question")
        y = torch.Tensor(y).long()
        
        return x, y

In [15]:
cont_answer_pairs = []
for ctx, quest in zip(list(df['context']), list(df['question'])):
  if 'float' in str(type(ctx)) or 'float' in str(type(quest)):
    continue
  else:
    cont_answer_pairs.append((ctx, quest))
len(cont_answer_pairs), cont_answer_pairs[0]

(86610,
 ("Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time.",
  'When did Beyonce start becoming popular?'))

In [16]:
dataset = WordData(cont_answer_pairs, word2index)

In [17]:
dataset.__getitem__(0)

(tensor([33972, 33404, 16932,    11,  3127,   270,  8242,    11, 36992,    11,
           200,    24,    23,   375,   437,   405,     2,  2852,    24,    14,
            28,   137,  2195,     2,  9315,     2,   380,  1913,     6,  2844,
             3,   375,     6,  1059,     7,  1971,     2,   736,     2,    66,
          1767,     7,   904,  4027,     6,  5264,  5557,    19,     8,   951,
             2,     6,   480,     5,  3101,     7,     1,   284,  2222,    19,
           406,  2195,     4,  1886,  1146,  1535,  1728,    11,   126,  9697,
            56,  1513,   951,     3,  1743,    21,    69,   623,     2, 23818]),
 tensor([   60,   116, 25922,   460,  1642,   803,   185,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]))

In [84]:
data_loader = DataLoader(dataset, batch_size=64, drop_last = True)
for x, y in data_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 80]) torch.Size([64, 20])


In [18]:
SOS_token = 0
EOS_token = 1


In [79]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, dropout_p=0.1, max_length=QUESTION_LENGTH, vectors=vectors):#output_size
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
#         self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.vectors = vectors
        self.vocab_size, self.embedding_dim = vectors.shape
        # self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vectors, device = device))
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(self.vectors))
#         self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, 80)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
#         self.out = nn.Linear(self.hidden_size, self.output_size)
        self.out = nn.Linear(self.hidden_size, self.vocab_size)

    def forward(self, input, hidden, encoder_outputs, debug = False):
#         embedded = self.embedding(input).view(1, 1, -1)
        if debug == True: 
            print("==forward_decoding_cycle===")
            print("input.shape", input.shape)
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)
        if debug == True: print("embedded[0].shape {}, hidden.shape {}".format(embedded[0].shape, hidden.shape))
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden), 1)), dim=1)
        if debug == True: 
            print("attn_weights.shape",attn_weights.shape,"encoder_outputs.shape", encoder_outputs.shape)
            print("attn_weights.unsqueeze(0).shape",attn_weights.unsqueeze(0).shape,"encoder_outputs.unsqueeze(0).shape", encoder_outputs.unsqueeze(0).shape)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        hidden = hidden.unsqueeze(0)
        
        output = F.relu(output)
        if debug == True: 
            print("output.shape {}, hidden.shape {}".format(output.shape,hidden.shape ))
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        hidden = hidden.squeeze(1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [80]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, vectors=vectors):#input_size
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.vectors = vectors
        self.vocab_size, self.embedding_dim = vectors.shape
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(self.vectors))
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first = True)

    def forward(self, input, hidden, debug = False):
        if debug == True: 
            print("====ENCODING_FORWARD====")
            print("input.shape", input.shape)
#         embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.embedding(input)
        output = embedded
        if debug == True: 
            print("embedded/output.shape",embedded.shape,"hidden.shape", hidden.shape  )
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 64, self.hidden_size, device=device)

In [60]:
class DecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, vectors):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.vectors = vectors
        self.vocab_size, self.embedding_dim = vectors.shape
        # self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(self.vectors))
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, self.vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, debug = False):
        if debug == True:
          print("===FORWARD_DECODER===")
          print("input.shape {}, hidden.shape {}".format(input.shape,hidden.shape ))
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [77]:
def train_easy_attention(input_tensor, target_batch_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, ctx_len =SENTENCE_LENGTH, 
          debug = False):
    if debug == True: print("target_batch_tensor.shape {}".format(target_batch_tensor.shape))
    

    input_length = input_tensor.size(0)
    target_length = target_batch_tensor.size(1)
    

    encoder_outputs_storage = torch.zeros(ctx_len, encoder.hidden_size, device=device)
    if debug == True: 
        print("target_length", target_length)
        print("encoder_outputs.shape", encoder_outputs.shape)
        print("input_length is", input_length)
        
    encoder_hidden = encoder.initHidden()
    encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)
    
    if debug == True: print("AFTER_ENCODING , encoder_output.shape{}, encoder_hidden.shape{}".format(encoder_output.shape, encoder_hidden.shape))
    
    for batch_element_index in range(64):
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        loss = 0
        target_tensor = target_batch_tensor[batch_element_index, :]
        target_tensor = target_tensor.unsqueeze(-1)
        if debug == True: 
            print("target_tensor.shape {}, target_tensor {}".format(target_tensor.shape, target_tensor))
            
        decoder_input = torch.tensor([[SOS_token]], device=device)
#         decoder_hidden = encoder_hidden[:,batch_element_index,:].unsqueeze(1)
        decoder_hidden = encoder_hidden[:,batch_element_index,:]
        
        encoder_output_in_current_index = encoder_output[batch_element_index,:,:]
        if debug == True: 
            print("go_to_decoder_with decoder_input{}, decoder_hidden.shape {}".format(decoder_input, decoder_hidden.shape))

        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if debug == True: 
                print("iterate_over_target {}th time, decoder_input.shape {}".format(di, decoder_input.shape))
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output_in_current_index)
            if debug == True: 
                print("AFTER_DECODER_STEP decoder_output.shape {}, decoder_input.shape {}, decoder_input {}".format(decoder_output.shape, decoder_input.shape, decoder_input))
#                 decoder_input = torch.tensor([decoder_input], device=device)
            target_word = torch.tensor([target_tensor[di]], device=device)
            loss += criterion(decoder_output, target_word)
            decoder_input = torch.tensor([target_tensor[di]], device=device).unsqueeze(1)  # Teacher forcing
#                 decoder_input = target_tensor[di]

        loss.backward(retain_graph=True)
        encoder_optimizer.step()
        decoder_optimizer.step()

    return loss.item() / target_length

In [22]:
def train_easy(input_tensor, target_batch_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, ctx_len =SENTENCE_LENGTH, 
          debug = False):
    if debug == True: print("target_batch_tensor.shape {}".format(target_batch_tensor.shape))
    

    input_length = input_tensor.size(0)
    target_length = target_batch_tensor.size(1)
    if debug == True: print("target_length", target_length)

    encoder_hidden = encoder.initHidden()
    encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)
    

    if debug == True: print("AFTER_ENCODING , encoder_output.shape{}, encoder_hidden.shape{}".format(encoder_output.shape, encoder_hidden.shape))
    
    for batch_element_index in range(64):
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        loss = 0
        target_tensor = target_batch_tensor[batch_element_index, :]
        if debug == True: 
            print("target_tensor.shape {}, target_tensor {}".format(target_tensor.shape, target_tensor))
            
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden[:,batch_element_index,:].unsqueeze(1)
        
        if debug == True: 
            print("go_to_decoder_with decoder_input{}, decoder_hidden.shape {}".format(decoder_input, decoder_hidden.shape))


        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if debug == True: 
                print("iterate_over_target {}th time, decoder_input.shape {}".format(di, decoder_input.shape))
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            if debug == True: 
                print("AFTER_DECODER_STEP decoder_output.shape {}, decoder_input.shape {}, decoder_input {}".format(decoder_output.shape, decoder_input.shape, decoder_input))
            target_word = torch.tensor([target_tensor[di]], device=device)
            loss += criterion(decoder_output, target_word)
            decoder_input = torch.tensor([target_tensor[di]], device=device).unsqueeze(1)  # Teacher forcing

        loss.backward(retain_graph=True)
        encoder_optimizer.step()
        decoder_optimizer.step()

    return loss.item() / target_length

In [87]:
def trainIters_loader(encoder, decoder, epochs = 5, learning_rate=0.01, total_items = len(cont_answer_pairs), debug = False, attention = False):
    plot_losses = []
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = []
    criterion = nn.NLLLoss()
    losses = []
    
    for n_epoch in range(epochs):
      progress_bar = tqdm(total=total_items, desc='Epoch {}'.format(n_epoch + 1))
      for input_tensor, target_tensor in data_loader:
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)
            if debug == True:print("input_tensor.shape", input_tensor.shape, "target_tensor.shape", target_tensor.shape)
            if attention == False:
                loss = train_easy(input_tensor, target_tensor, encoder,
                          decoder, encoder_optimizer, decoder_optimizer, criterion)
            else:
                loss = train_easy_attention(input_tensor, target_tensor, encoder,
                          decoder, encoder_optimizer, decoder_optimizer, criterion)
            losses.append(loss)
            progress_bar.set_postfix(train_loss = np.mean(losses[-100:]))
            progress_bar.update(input_tensor.shape[0])
            print_loss_total += loss
            
            
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)
            if debug == True:print("input_tensor.shape", input_tensor.shape, "target_tensor.shape", target_tensor.shape)
            if attention == False:
                print("no_attent")
                loss = train_easy(input_tensor, target_tensor, encoder,
                          decoder, encoder_optimizer, decoder_optimizer, criterion)
            else:
                print("attent")
                loss = train_easy_attention(input_tensor, target_tensor, encoder,
                          decoder, encoder_optimizer, decoder_optimizer, criterion)
            losses.append(loss)
            progress_bar.set_postfix(train_loss = np.mean(losses[-100:]))
            progress_bar.update(input_tensor.shape[0])
            print_loss_total += loss

    print_loss_avg = print_loss_total / total_items
    # tqdm.write('Losses: train - {:.3f}, test = {:.3f}'.format(np.mean(train_losses), mean_test_loss))
    tqdm.write('Losses: train - {:.3f}'.format(print_loss_avg))

In [24]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

# TRAIN WITHOUT ATTENTION

In [85]:
hidden_size = 50
# 37487
encoder1 = EncoderRNN(hidden_size)
encoder1.to(device)
# attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, max_length = MAX_LENGTH, dropout_p=0.1).to(device)
# attn_decoder1 = AttnDecoderRNN(hidden_size, dropout_p=0.1)
attn_decoder1 = DecoderRNN(hidden_size, vectors)
attn_decoder1.to(device)



DecoderRNN(
  (embedding): Embedding(64902, 50)
  (gru): GRU(50, 50)
  (out): Linear(in_features=50, out_features=64902, bias=True)
  (softmax): LogSoftmax()
)

In [88]:
trainIters_loader(encoder1, attn_decoder1)

KeyboardInterrupt: 

In [29]:
torch.save(encoder1, 'Encoder_no_attention')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type EncoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [30]:
torch.save(attn_decoder1, 'Decoder_no_attention_5_epochs')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LogSoftmax. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# TRAIN WITH ATTENTION

In [83]:
hidden_size = 50
# encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attanetion_encoder = EncoderRNN(hidden_size)
attanetion_encoder.to(device)
real_attention_decoder = AttnDecoderRNN(hidden_size, dropout_p=0.1)
real_attention_decoder.to(device)

trainIters_loader(attanetion_encoder, real_attention_decoder, attention = True)

KeyboardInterrupt: 